## Setup

In [1]:
#Based on tutorial found at: https://www.tensorflow.org/tutorials/text/text_generation

#Importing required packages
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
#Reading in previously scraped text
text = ''
text += open('reddit_scrape.txt', 'rb').read().decode(encoding='utf-8')

#Provide information about scraped dataset
print ('Length of text: {} characters'.format(len(text)))
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

Length of text: 22443 characters
81 unique characters


## Process The Text

In [3]:
#Vectorizing text
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

In [4]:
#Creating training examples
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [5]:
#Method to duplicate and shift to form input and target
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [6]:
#Creating training batches
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

## Build the Model

In [7]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [8]:
# The function to build the RNN model
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

In [9]:
#Calling build_model with all of our previously defined parameters
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

## Train the Model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

In [10]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
    #return tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)

model.compile(optimizer = tf.train.AdamOptimizer(), loss = loss)

In [11]:
# Directory where the checkpoints will be saved
checkpoint_dir = './rnn_ckpt'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True)

In [12]:
# Epochs indicate the amount of times all of the training vectors are used to update the weights in your model
# The lower you set this number the faster the next cell will run, but you may find your model's performance lacking
# There are diminishing returns with a large amount of epochs so the solution isn't neccessarily to set EPOCHS = int_max
# Try experimenting with different amounts
# The way this model is designed you can load the checkpoint from any epoch if you experience diminishing returns in loss
EPOCHS=256

In [13]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/256
3/3 [==============================] - 3s 1s/step - loss: 4.3388
Epoch 2/256
3/3 [==============================] - 1s 266ms/step - loss: 7.0323
Epoch 3/256
3/3 [==============================] - 1s 275ms/step - loss: 4.0463
Epoch 4/256
3/3 [==============================] - 1s 483ms/step - loss: 4.1246
Epoch 5/256
3/3 [==============================] - 1s 361ms/step - loss: 4.0662
Epoch 6/256
3/3 [==============================] - 1s 291ms/step - loss: 3.9142
Epoch 7/256
3/3 [==============================] - 1s 379ms/step - loss: 3.7226
Epoch 8/256
3/3 [==============================] - 1s 229ms/step - loss: 3.5006
Epoch 9/256
3/3 [==============================] - 1s 315ms/step - loss: 3.1673
Epoch 10/256
3/3 [==============================] - 0s 139ms/step - loss: 3.0503
Epoch 11/256
3/3 [==============================] - 1s 210ms/step - loss: 2.9955
Epoch 12/256
3/3 [==============================] - 1s 304ms/step - loss: 2.9215
Epoch 13/256
3/3 [======================

In [14]:
tf.train.latest_checkpoint(checkpoint_dir)

'./rnn_ckpt/ckpt_256'

In [15]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            20736     
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 81)             83025     
Total params: 4,042,065
Trainable params: 4,042,065
Non-trainable params: 0
_________________________________________________________________


## Generate Text

In [16]:
# num_generate is the number of characters you'd like to generate
# model is the model we've defined, this is simply passed in as "model"
# start_string is what kicks off our RNN, setting an initial state to predict from
def generate_text(num_generate, model, start_string):
  # Evaluation step (generating text using the learned model)

  # Converting our start string to numbers (vectorizing) 
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
    text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
    temperature = 1.0

  # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
      # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
      
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [17]:
generate_text(250, model, start_string=u"[WP]")

Instructions for updating:
Use `tf.random.categorical` instead.


'[WP]IFOR] AT THE MOON". You plepsed and takes over and Jack drives home at night, and accidentally runs over another Jack, a plumber. The Skviors round years, mice for on the apocollpscellice you lan a lifetime spacily the lang smowary rearons. Aporay of'

In [ ]:
file = open("Generated_reddit.txt","a")
file.write(generate_text(65000, model, start_string=u"[WP]"))
file.close()

In [18]:
file = open("reddit_posts.txt","a")
i=0
while i < 25:
    text = generate_text(random.randint(30,250), model, start_string=u"[WP]")
    text = text + '\n'
    file.write(text)
    i+=1

file.close()

NameError: name 'random' is not defined